# VIS Tutorial 

When starting an image-based phenotyping project it is important to consider what the end goals of the project are.
This is important because the goals of the project will determine the the camera type, imaging layout, and will help to 
guide downstream analysis. For example, if the goal of the project is to quantify the growth rates of a population of 
Arabidopsis plants, you may want to take timelapse images of whole flats of plants with an RGB (VIS) camera.

To run a VIS pipeline over a single VIS image there are two required inputs:

1.  **Image:** Images can be processed regardless of what type of VIS camera was used (high-throughput platform, digital camera, cell phone camera).
Image processing will work with adjustments if images are well lit and free of background that is similar in color to plant material.  
2.  **Output directory:** If debug mode is set to 'print' output images from each step are produced.

In [7]:
# Import PlantCV 
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/vis/original_image.jpg"
        self.debug = "plot"
        self.writeimg= False
        self.result = "./vis_tutorial_results"
        self.outdir = "."
        
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image
img, path, filename = pcv.readimage(args.image)

In [ ]:
# Convert RGB to HSV and extract the saturation channel

# Inputs:
#   rgb_image - RGB image data 
#   channel - Split by 'h' (hue), 's' (saturation), or 'v' (value) channel
s = pcv.rgb2gray_hsv(img, 's')

In [ ]:
# Threshold can be on either light or dark objects in the image. 

# Inputs:
#   gray_img - Grayscale image data 
#   threshold- Threshold value (between 0-255)
#   max_value - Value to apply above threshold (255 = white) 
#   object_type - 'light' (default) or 'dark'. If the object is lighter than the background then standard threshold is done.
#                 If the object is darker than the background then inverse thresholding is done. 
s_thresh = pcv.threshold.binary(s, 85, 255, 'light')

In [ ]:
# Median Blur 

# Inputs: 
#   gray_img - Grayscale image data 
#   ksize - Kernel size (integer or tuple), (ksize, ksize) box if integer input,
#           (n, m) box if tuple input 
s_mblur = pcv.median_blur(s_thresh, 5)
s_cnt = pcv.median_blur(s_thresh, 5)

In [ ]:
# An alternative to using median_blur is gaussian_blur, which applies 
# a gaussian blur filter to the image. Depending on the image, one 
# technique may be more effective than others. 

# Inputs:
#   img - RGB or grayscale image data
#   ksize - Tuple of kernel size
#   sigmax - Standard deviation in X direction; if 0 (default), 
#            calculated from kernel size
#   sigmay - Standard deviation in Y direction; if sigmaY is 
#            None (default), sigmaY is taken to equal sigmaX
gaussian_img = pcv.gaussian_blur(img=s_thresh, ksize=(5, 5), sigmax=0, sigmay=None)

In [ ]:
# Convert RGB to LAB and extract the blue channel ('b')

# Input:
#   rgb_img - RGB image data 
#   channel- Split by 'l' (lightness), 'a' (green-magenta), or 'b' (blue-yellow) channel
b = pcv.rgb2gray_lab(img, 'b')

# Threshold the blue channel image 

b_thresh = pcv.threshold.binary(b, 160, 255, 'light')
b_cnt = pcv.threshold.binary(b, 160, 255, 'light')

In [ ]:
# Join the threshold saturation and blue-yellow images

# Inputs: 
#   bin_img1 - Binary image data to be compared to bin_img2
#   bin_img2 - Binary image data to be compared to bin_img1
bs = pcv.logical_or(s_mblur, b_cnt)

In [ ]:
# Appy Mask (for VIS images, mask_color='white')

# Inputs:
#   rgb_img - RGB image data 
#   mask - Binary mask image data 
#   mask_color - 'white' or 'black' 
masked = pcv.apply_mask(img, bs, 'white')

Now we'll focus on capturing the plant in the masked image. We will use masked green-magenta and blue-yellow channels. 
Then two channels are thresholded to caputre different portions of the plant, and thre three images are joined together. 
Small objected are filled, and the resulting binary image is used to mask the masked image previously obtained. 

In [ ]:
# Convert RGB to LAB and extract the Green-Magenta and Blue-Yellow channels

masked_a = pcv.rgb2gray_lab(masked, 'a')
masked_b = pcv.rgb2gray_lab(masked, 'b')

In [ ]:
# Threshold the green-magenta and blue images

maskeda_thresh = pcv.threshold.binary(masked_a, 115, 255, 'dark')
maskeda_thresh1 = pcv.threshold.binary(masked_a, 135, 255, 'light')
maskedb_thresh = pcv.threshold.binary(masked_b, 128, 255, 'light')

In [ ]:
# Join the thresholded saturation and blue-yellow images (OR)

ab1 = pcv.logical_or(maskeda_thresh, maskedb_thresh)
ab = pcv.logical_or(maskeda_thresh1, ab1)

In [ ]:
# Depending on the situation it might be useful to use the 
# exclusive or (pcv.logical_xor) function. 

# Inputs: 
#   bin_img1 - Binary image data to be compared to bin_img2
#   bin_img2 - Binary image data to be compared to bin_img1
xor_img = pcv.logical_xor(maskeda_thresh, maskedb_thresh)

In [ ]:
# Fill small objects (reduce image noise) 

# Inputs: 
#   bin_img - Binary image data 
#   size - Minimum object area size in pixels (must be an integer), and smaller objects will be filled
ab_fill = pcv.fill(ab, 200)

In [ ]:
# Apply mask (for VIS images, mask_color=white)

masked2 = pcv.apply_mask(masked, ab_fill, 'white')

Now we need to identify the objects (also called contours) within the image 

In [ ]:
# Identify objects

# Inputs: 
#   img - RGB or grayscale image data for plotting 
#   mask - Binary mask used for detecting contours 
id_objects, obj_hierarchy = pcv.find_objects(masked2, ab_fill)

In [ ]:
# Define the region of interest (ROI) 

# Inputs: 
#   x - The x-coordinate of the upper left corner of the rectangle 
#   y - The y-coordinate of the upper left corner of the rectangle 
#   h - The height of the rectangle 
#   w - The width of the rectangle 
#   img - RGB or grayscale image to plot the ROI on 
roi1, roi_hierarchy= pcv.roi.rectangle(x=100, y=100, h=200, w=200, img=masked2)

In [ ]:
# Decide which objects to keep

# Inputs:
#   img - RGB or grayscale image data to display kept objects on 
#   roi_type - 'cutto' or 'partial' => include objects that are partially inside or overlapping with the ROI 
#   roi_contour - contour of ROI, output from pcv.roi.rectangle in this case
#   object_contour - Contour of objects, output from pcv.roi.rectangle in this case 
#   obj_hierarchy - Hierarchy of objects, output from pcv.find_objects function 
roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(img, 'partial', roi1, roi_hierarchy, id_objects, obj_hierarchy)

In [ ]:
# Object combine kept objects

# Inputs:
#   img - RGB or grayscale image data for plotting 
#   contours - Contour list 
#   hierarchy - Contour hierarchy array 
obj, mask = pcv.object_composition(img, roi_objects, hierarchy3)

The next step is to analyze the plant object for traits such as horizontal height, shape, or color.

In [ ]:
############### Analysis ################ 
  
# Find shape properties, output shape image (optional)

# Inputs:
#   img - RGB or grayscale image data 
#   obj- Single or grouped contour object
#   mask - Binary image mask to use as mask for moments analysis 
# Returns:
#   shape_header, shape_data, and analysis_images (an array containing the original image with shape
#   data plotted on it, and the mask) 
shape_header, shape_data, analysis_images = pcv.analyze_object(img, obj, mask)

# analysis_images contains two images, the original image with shape data drawn on, and the mask
shape_info_img, shape_mask = analysis_images
# Plot them out 
pcv.plot_image(shape_info_img)
pcv.plot_image(shape_mask)

In [ ]:
# Shape properties relative to user boundary line (optional)

# Inputs:
#   img - RGB or grayscale image data 
#   obj - Single or grouped contour object 
#   mask - Binary mask of selected contours 
#   line_position - Position of boundary line (a value of 0 would draw a line 
#                   through the bottom of the image) 
boundary_header, boundary_data, boundary_images = pcv.analyze_bound_horizontal(img, obj, mask, 1680)

# There are two images returned in boundary_images. Plot them both out 
white_background_horiz_img, horiz_bound_img = boundary_images
pcv.plot_image(white_background_horiz_img)
pcv.plot_image(horiz_bound_img)

In [ ]:
# Determine color properties: Histograms, Color Slices and Pseudocolored Images, output color analyzed images (optional)

# Inputs:
#   rgb_img - RGB image data
#   mask - Binary mask of selected contours 
#   bins - Number of color bins (0-256)
#   hist_plot_type - None (default), 'all', 'rgb', 'lab', or 'hsv'
#                    This is the data to be printed to the SVG histogram file  
color_header, color_data, color_histogram = pcv.analyze_color(img, kept_mask, 256, 'all')

# Since color_histogram is a plotnine ggplot we can plot it to the screen without pcv.plot_image
color_histogram.save(os.path.join(params.debug_outdir, 'analyze_color_hist.png'))

In [ ]:
# Divide plant object into twenty equidistant bins and assign pseudolandmark points based upon their 
# actual (not scaled) position. Once this data is scaled this approach may provide some information 
# regarding shape independent of size.

top_x, bottom_x, center_v_x = pcv.x_axis_pseudolandmarks(obj, mask, img)

top_y, bottom_y, center_v_y = pcv.y_axis_pseudolandmarks(obj, mask, img)

In [ ]:
# Write shape and color data to results file
# Result file will go to the notebooks folder! 
result = open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
for row in shape_img:  
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.write('\t'.join(map(str,color_header)))
result.write("\n")
result.write('\t'.join(map(str,color_data)))
result.write("\n")
for row in color_img:
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.close()